# Importing Library


In [28]:
import scipy.io
import pandas as pd
import numpy as np
import scipy.io
import os
from itertools import chain

# Extracting File Path


In [11]:
# Common part of the file paths
base_path = r'file path'

# List of rpm values
rpm_values = [1797, 1772, 1750,1730]

# Create an empty list to store file paths
normal_bearing_data_file_paths = []

# Iterate over rpm values and create file paths
for i, rpm in enumerate(rpm_values, start=97):
    folder_name = f'load-hp-{i-97}-rpm-{rpm}'
    file_name = f'{i}.mat'
    file_path = os.path.join(base_path, folder_name, file_name)
    normal_bearing_data_file_paths.append(file_path)

In [47]:
# Common part of the file paths
base_path = r'file path'

# Different parts for the file names
file_name_parts = ["B007", "B014", "B021","B028","IR007","IR014","IR021","IR028","OR007@3","OR007@6","OR007@12","OR014@6","OR021@3","OR021@6","OR021@12"]

# Generate file paths using nested loops
all_fault_bearing_data_file_paths = []

for part in file_name_parts:
    fault_bearing_data_file_paths = []
    
    for i, rpm in enumerate([1797, 1772, 1750, 1730]):
        file_path = os.path.join(base_path, f'load-hp-{i}-rpm-{rpm}', f'historic_{part}_{i}.mat')
        fault_bearing_data_file_paths.append(file_path)
    
    all_fault_bearing_data_file_paths.append(fault_bearing_data_file_paths)

# Flatten the nested list
all_file_paths_flat = list(chain(*all_fault_bearing_data_file_paths))



In [50]:
# Combine the two lists of file paths
all_file_paths = normal_bearing_data_file_paths + all_file_paths_flat
print(len(all_file_paths))
# Print the resulting file paths
for path in all_file_paths:
    print(path)


64
C:\Users\PARAM M. SURELIYA\Desktop\normal-baseline-data\normal-baseline-data\load-hp-0-rpm-1797\97.mat
C:\Users\PARAM M. SURELIYA\Desktop\normal-baseline-data\normal-baseline-data\load-hp-1-rpm-1772\98.mat
C:\Users\PARAM M. SURELIYA\Desktop\normal-baseline-data\normal-baseline-data\load-hp-2-rpm-1750\99.mat
C:\Users\PARAM M. SURELIYA\Desktop\normal-baseline-data\normal-baseline-data\load-hp-3-rpm-1730\100.mat
C:\Users\PARAM M. SURELIYA\Desktop\historic-12k-Drive-End-Bearing-Fault-Data\historic-12k-Drive-End-Bearing-Fault-Data\load-hp-0-rpm-1797\historic_B007_0.mat
C:\Users\PARAM M. SURELIYA\Desktop\historic-12k-Drive-End-Bearing-Fault-Data\historic-12k-Drive-End-Bearing-Fault-Data\load-hp-1-rpm-1772\historic_B007_1.mat
C:\Users\PARAM M. SURELIYA\Desktop\historic-12k-Drive-End-Bearing-Fault-Data\historic-12k-Drive-End-Bearing-Fault-Data\load-hp-2-rpm-1750\historic_B007_2.mat
C:\Users\PARAM M. SURELIYA\Desktop\historic-12k-Drive-End-Bearing-Fault-Data\historic-12k-Drive-End-Bearing-Fa

# Data Labelling 

In [91]:
# Create an empty list to store DataFrames
all_dataframes = []

# Iterate over each file path
for data_path in all_file_paths:
    # Load data from the MAT file
    data_dict = scipy.io.loadmat(data_path)

    # Extract titles from the keys of the dictionary
    titles = list(data_dict.keys())


    # Create an empty list to store DataFrames for the current data file
    dataframes = []

    # Iterate over titles and create DataFrames
    for title in titles:
        # Check if the value associated with the key is a NumPy array
        if isinstance(data_dict[title], np.ndarray):
            # Extract the array corresponding to the title
            array = data_dict[title]

            # Create a DataFrame for the array with the column name as the title
            df = pd.DataFrame(array, columns=[title])

            # Append the DataFrame to the list for the current data file
            dataframes.append(df)

    # Combine the DataFrames for the current data file into a single DataFrame
    result_df = pd.concat(dataframes, axis=1)
    
    # Transpose the resulting DataFrame
    result_df_transposed = result_df.transpose()

    # Append the resulting DataFrame to the list of all DataFrames
    all_dataframes.append(result_df_transposed)

# Concatenate all DataFrames into a single DataFrame
final_result = pd.concat(all_dataframes, axis=0)

# List of index labels to remove
rows_to_remove = ['ans', 'X097RPM', 'X100RPM', 'X118RPM', 'X119RPM', 'X120RPM', 'X121RPM','X185RPM','X186RPM','X187RPM','X188RPM','X222RPM','X223RPM','X224RPM','X225RPM','X105RPM','X106RPM','X107RPM','X108RPM','X169RPM','X170RPM','X171RPM','X172RPM','X209RPM','X210RPM','X211RPM','X212RPM','X144RPM','X145RPM','X146RPM','X147RPM','X130RPM','X131RPM','X132RPM','X133RPM','X156RPM','X158RPM','X159RPM','X160RPM','X197RPM','X198RPM','X199RPM','X200RPM','X246RPM','X247RPM','X248RPM','X249RPM','X234RPM','X235RPM','X236RPM','X237RPM','X258RPM','X259RPM','X260RPM','X261RPM']

# Remove rows with specified index labels
final_result = final_result.drop(rows_to_remove, errors='ignore')

#Removing duplicate row named X098_FE_time and X098_DE_time
final_result = final_result.loc[~final_result.index.duplicated(keep='first')]

# Add a new column with the specified pattern of values
final_result.insert(0, 'Label', [0] * 8 + [1] * 12 + [2] * 12 + [3] * 12 + [4] * 4 + [5] * 12 + [6] * 12 + [7] * 12 + [8] * 4 + [9] * 36 + [10] * 12 + [11] * 36)


# Save DataFrame to CSV file
#final_result.to_csv('raw_data.csv', index=True)
display(final_result)

,Label,0,1,2,3,4,5,6,7,8,...,485633,485634,485635,485636,485637,485638,485639,485640,485641,485642
X097_DE_time,0,0.053197,0.088662,0.099718,0.058621,-0.004590,-0.056952,-0.071764,-0.058621,-0.046521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X097_FE_time,0,0.145667,0.097796,0.054856,0.036982,0.054445,0.021162,-0.003698,-0.010684,0.029380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X098_DE_time,0,0.046104,-0.037134,-0.089496,-0.084906,-0.038594,0.025451,0.040680,0.042558,0.038177,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X098_FE_time,0,0.023216,0.081155,0.095331,0.091016,0.038625,0.027120,0.050131,0.045816,0.014176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X099_DE_time,0,0.064254,0.063002,-0.004381,-0.035882,-0.023991,0.005215,0.030249,0.007510,-0.016481,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X260_FE_time,11,-0.000205,-0.125738,-0.020545,0.001233,-0.169500,0.212440,0.010067,-0.069444,0.151625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X260_BA_time,11,0.096409,0.010703,-0.020078,-0.087396,-0.108118,0.005754,-0.033598,-0.071542,-0.009013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X261_DE_time,11,0.045076,0.117765,0.031269,-0.089339,0.051979,0.148222,0.140912,0.013401,-0.032487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X261_FE_time,11,0.118753,-0.079716,0.094920,0.162309,-0.124711,0.078484,0.059376,-0.001438,0.161693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Preparation

In [93]:
# Count non-NaN values in each row
non_nan_count_per_row = final_result.count(axis=1)

# Create a new DataFrame from the Series
count_df = pd.DataFrame({'Non-NaN Count': non_nan_count_per_row})

# Display the new DataFrame
display(count_df)

,Non-NaN Count
X097_DE_time,243939
X097_FE_time,243939
X098_DE_time,483904
X098_FE_time,483904
X099_DE_time,485064
...,...
X260_FE_time,100001
X260_BA_time,100001
X261_DE_time,100001
X261_FE_time,100001


In [96]:
k=0
result_df1 = pd.DataFrame()
result_df = pd.DataFrame()
# Using iterrows to iterate through rows
for index, row in final_result.iterrows(): 
    result_df = pd.DataFrame()
    # Accessing the first row
    first_row = row
    # Remove NaN values from the first row
    cleaned_first_row = first_row.dropna()
    # Extract the first element of cleaned_first_row
    first_element = cleaned_first_row.iloc[0]
    # Remove the first element from the cleaned first row
    cleaned_first_row = cleaned_first_row.iloc[1:]
    #display(cleaned_first_row)


    i = 2048
    diff = 1434
    values_to_append = cleaned_first_row.iloc[:2048]
    result_df = pd.concat([result_df, values_to_append], axis=1)
    i = i + diff

    while i < count_df.iloc[k,0]:
        # Take the first 2048 values from the single column
        values_to_append = cleaned_first_row.iloc[i - 2048:i].reset_index(drop=True)
        # Append the values to the result DataFrame
        result_df = pd.concat([result_df, values_to_append], axis=1)

        i = i + diff
    result_df=result_df.T
    new_column_name = 'Label'
    result_df[new_column_name] = first_element

    # Move the new column to the front
    result_df = pd.concat([result_df[new_column_name], result_df.drop(columns=[new_column_name])], axis=1)
    result_df = result_df.T
    
    result_df1 = pd.concat([result_df1, result_df], axis=1)
    k=k+1
    # Transpose the result DataFrame if needed
result_df1=result_df1.T
# Display the result DataFrame
display(result_df1)


,Label,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
X097_DE_time,0.0,0.053197,0.088662,0.099718,0.058621,-0.004590,-0.056952,-0.071764,-0.058621,-0.046521,...,-0.039637,-0.017315,-0.023574,-0.034004,-0.044852,-0.038802,-0.016689,0.000209,-0.021905,-0.071138
X097_DE_time,0.0,-0.054449,-0.035465,-0.011474,0.013560,0.068634,0.118911,0.145822,0.120162,0.039428,...,0.034004,0.059664,0.078857,0.097006,0.121414,0.160425,0.177532,0.153541,0.095129,0.026077
X097_DE_time,0.0,0.012100,-0.046313,-0.103473,-0.118911,-0.086158,-0.041723,-0.011474,-0.005841,0.020653,...,0.054031,-0.028789,-0.104308,-0.149160,-0.147908,-0.128090,-0.072181,-0.017524,0.034004,0.091791
X097_DE_time,0.0,0.009179,-0.034213,-0.038802,-0.016272,-0.000834,-0.006676,-0.001043,0.025660,0.042558,...,0.006676,-0.040263,-0.057786,-0.032127,-0.007510,0.007719,0.008762,0.019610,0.046730,0.079691
X097_DE_time,0.0,0.125586,0.157505,0.167727,0.116199,0.035882,-0.072390,-0.164806,-0.182747,-0.154167,...,0.115573,0.131010,0.136852,0.110983,0.037551,-0.056535,-0.119328,-0.118494,-0.066548,-0.012726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X261_BA_time,11.0,-0.009053,-0.031466,-0.054803,0.018228,0.056373,0.024907,0.087355,0.097334,0.032230,...,-0.311478,-0.352681,-0.377467,-0.214666,0.040801,0.221547,0.250075,0.135359,0.153988,0.198612
X261_BA_time,11.0,-0.059793,0.001006,0.053355,0.076894,0.048325,-0.036455,-0.039996,-0.045951,-0.045187,...,0.059913,0.058666,-0.011830,0.020481,-0.036053,-0.091218,-0.012071,-0.031506,-0.071462,-0.012836
X261_BA_time,11.0,0.041203,0.003340,0.153787,0.012594,-0.203762,-0.298320,-0.228548,0.035892,0.106026,...,0.028287,0.040358,0.007645,-0.023941,-0.034564,-0.029212,-0.011025,-0.009295,-0.038708,-0.015089
X261_BA_time,11.0,-0.015894,0.004064,0.061845,-0.007967,0.020964,0.063736,0.024303,0.072226,0.047198,...,0.073031,0.042249,0.056051,0.084297,0.067599,0.117372,0.079026,-0.004547,-0.003179,-0.031586


# Saving DataFrame to CSV file

In [97]:
# Save DataFrame to CSV file
result_df1.to_csv('labelled_data.csv', index=True)